# RAG 시스템의 재정렬(Reranking) 방법

## 개요
재정렬은 **검색 기반 생성(Retrieval-Augmented Generation, RAG)** 시스템에서 검색된 문서의 관련성과 품질을 향상시키기 위한 중요한 단계입니다. 초기 검색된 문서를 재평가하고 순서를 재조정하여, 생성 또는 프레젠테이션에 가장 적합한 정보를 우선시할 수 있도록 하는 과정입니다.

## 동기
RAG 시스템에서 재정렬을 사용하는 주된 이유는 초기 검색 방법의 한계를 극복하기 위함입니다. 초기 검색 방법은 간단한 유사도 측정에 의존하는 경우가 많아, 쿼리와 문서 간의 미묘한 관계를 파악하지 못할 수 있습니다. 재정렬 과정을 통해 보다 정교한 관련성 평가를 수행하고, 이를 통해 생성 단계에서 가장 적절한 정보가 활용될 수 있도록 합니다.

## 주요 구성 요소
재정렬 시스템은 일반적으로 다음과 같은 구성 요소를 포함합니다:

1. **초기 검색기**: 보통 임베딩 기반의 유사도 검색을 수행하는 벡터 스토어.
2. **재정렬 모델**: 다음 중 하나를 사용하여 관련성을 평가합니다.
   - 대규모 언어 모델(LLM)을 사용한 관련성 점수 부여
   - 관련성 평가에 특화된 Cross-Encoder 모델
3. **점수 매기기 메커니즘(Scoring Mechanism)**: 문서에 관련성 점수를 부여하는 방법.
4. **정렬 및 선택 로직**: 새로운 점수를 기준으로 문서를 재정렬하고 상위 문서를 선택하는 로직.

## 방법론 세부 사항
재정렬 과정은 일반적으로 다음 단계로 진행됩니다:

1. **초기 검색**: 초기에는 잠재적으로 관련이 있는 문서를 검색하여 가져옵니다.
2. **쌍 생성**: 각 검색된 문서와 쿼리의 쌍을 만듭니다.
3. **점수 매기기**:
   - **LLM 방법**: LLM에 프롬프트를 사용하여 문서의 관련성을 평가하도록 합니다.
   - **Cross-Encoder 방법**: 쿼리-문서 쌍을 모델에 직접 입력하여 관련성을 평가합니다.
4. **점수 해석**: 관련성 점수를 구문 분석하고 정규화합니다.
5. **재정렬**: 새로운 관련성 점수에 따라 문서를 정렬합니다.
6. **선택**: 재정렬된 리스트에서 상위 K개의 문서를 선택합니다.

## 이 접근법의 장점
재정렬은 여러 가지 이점을 제공합니다:

1. **관련성 향상**: 더 정교한 모델을 사용함으로써, 미세한 관련성 요소까지 포착하여 높은 관련성을 보장합니다.
2. **유연성**: 특정 요구사항과 자원에 따라 다양한 재정렬 방법을 적용할 수 있습니다.
3. **문맥 품질 향상**: RAG 시스템에 더 관련성 높은 문서를 제공함으로써 생성 응답의 품질을 높입니다.
4. **잡음 감소**: 덜 관련성 높은 정보를 필터링하고, 가장 적합한 콘텐츠에 집중하게 합니다.

## 결론
재정렬은 RAG 시스템에서 검색된 정보의 품질을 크게 향상시키는 강력한 기법입니다. LLM 기반 점수 부여 또는 특화된 Cross-Encoder 모델을 사용하더라도, 재정렬은 문서의 관련성을 더 정밀하고 정확하게 평가할 수 있게 합니다. 이는 곧 후속 작업의 성능 향상으로 이어지며, 고급 RAG 구현에서 필수적인 구성 요소로 작용합니다.

LLM 기반과 Cross-Encoder 재정렬 방법 중에서 선택할 때는 정확도 요구 사항, 가용한 컴퓨팅 자원, 그리고 특정 응용 프로그램의 필요 사항을 고려할 필요가 있습니다. 두 접근 방식 모두 기본 검색 방법보다 상당한 개선을 제공하며, RAG 시스템의 전체적인 효율성을 높이는 데 기여합니다.


<div style="text-align: center;">

<img src="../images/reranking-visualization.svg" alt="rerank llm" style="width:100%; height:auto;">
</div>

<div style="text-align: center;">

<img src="../images/reranking_comparison.svg" alt="rerank llm" style="width:100%; height:auto;">
</div>

### Import relevant libraries

In [49]:
import os
import sys
from dotenv import load_dotenv
from langchain.docstore.document import Document
from typing import List, Dict, Any, Tuple
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_core.retrievers import BaseRetriever
from sentence_transformers import CrossEncoder


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks
from helper_functions import *
from evaluation.evalute_rag import *

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')

In [50]:
# ! pip install sentence-transformers
# ! pip install tf-keras



### Define the document's path

In [51]:
path = "../data/Understanding_Climate_Change.pdf"

### Create a vector store

In [52]:
vectorstore = encode_pdf(path)

## Method 1: LLM based function to rerank the retrieved documents

<div style="text-align: center;">

<img src="../images/rerank_llm.svg" alt="rerank llm" style="width:40%; height:auto;">
</div>

### Create a custom reranking function


In [53]:
import os
import sys
from dotenv import load_dotenv
from langchain.docstore.document import Document
from typing import List, Dict, Any, Tuple
from langchain_openai import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain_core.retrievers import BaseRetriever
from sentence_transformers import CrossEncoder


sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..'))) # Add the parent directory to the path sicnce we work with notebooks
from helper_functions import *
from evaluation.evalute_rag import *

# Load environment variables from a .env file
load_dotenv()

# Set the OpenAI API key environment variable
os.environ["OPENAI_API_KEY"] = os.getenv('OPENAI_API_KEY')
class RatingScore(BaseModel):
    relevance_score: float = Field(..., description="The relevance score of a document to a query.")

# 쿼리와 문서가 주어졌을 때 둘의 관련성을 점수로 파악하는 llm 
def rerank_documents(query: str, docs: List[Document], top_n: int = 3) -> List[Document]:
    prompt_template = PromptTemplate(
        input_variables=["query", "doc"],
        template="""On a scale of 1-10, rate the relevance of the following document to the query. Consider the specific context and intent of the query, not just keyword matches.
        Query: {query}
        Document: {doc}
        Relevance Score:"""
    )
    
    llm = ChatOpenAI(temperature=0, model_name="gpt-4o", max_tokens=4000)
    llm_chain = prompt_template | llm.with_structured_output(RatingScore)
    


    # 점수가 매겨지고, 점수 순서대로 리랭크 되어 top n 만큼 문서가 출력된다. 
    scored_docs = []
    for doc in docs:
        input_data = {"query": query, "doc": doc.page_content}
        score = llm_chain.invoke(input_data).relevance_score
        try:
            score = float(score)
        except ValueError:
            score = 0  # Default score if parsing fails
        scored_docs.append((doc, score))
    
    reranked_docs = sorted(scored_docs, key=lambda x: x[1], reverse=True)
    return [doc for doc, _ in reranked_docs[:top_n]]

### Example usage of the reranking function with a sample query relevant to the document


In [34]:
query = "What are the impacts of climate change on biodiversity?"
# 초기 문서 반환 
initial_docs = vectorstore.similarity_search(query, k=15)
# 초기 문서를 리랭크 
reranked_docs = rerank_documents(query, initial_docs)

# print first 3 initial documents
print("Top initial documents:")
for i, doc in enumerate(initial_docs[:3]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")  # Print first 200 characters of each document


# Print results
print(f"Query: {query}\n")
print("리랭크된 문서:")
for i, doc in enumerate(reranked_docs):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")  # Print first 200 characters of each document

Top initial documents:

Document 1:
Climate change is altering terrestrial ecosystems by shifting habitat ranges, changing species 
distributions, and impacting ecosystem functions. Forests, grasslands, and deserts are 
experiencing shi...

Document 2:
goals. Policies should promote synergies between biodiversity conservation and climate 
action.  
Chapter 10: Climate Change and Human Health  
Health Impacts  
Heat -Related Illnesses  
Rising temper...

Document 3:
managed retreats.  
Extreme Weather Events  
Climate change is linked to an increase in the frequency and severity of extreme weather 
events, such as hurricanes, heatwaves, droughts, and heavy rainfa...
Query: What are the impacts of climate change on biodiversity?

리랭크된 문서:

Document 1:
Climate change is altering terrestrial ecosystems by shifting habitat ranges, changing species 
distributions, and impacting ecosystem functions. Forests, grasslands, and deserts are 
experiencing shi...

Document 2:
protection, and habitat

### Create a custom retriever based on our reranker

In [45]:
# Create a custom retriever class
class CustomRetriever(BaseRetriever, BaseModel):
    
    vectorstore: Any = Field(description="Vector store for initial retrieval")

    class Config:
        arbitrary_types_allowed = True

    def get_relevant_documents(self, query: str, num_docs=2) -> List[Document]:
        initial_docs = self.vectorstore.similarity_search(query, k=30)
        return rerank_documents(query, initial_docs, top_n=num_docs)


# Create the custom retriever
custom_retriever = CustomRetriever(vectorstore=vectorstore)

# Create an LLM for answering questions
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

# Create the RetrievalQA chain with the custom retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=custom_retriever,
    return_source_documents=True
)


### Example query


In [46]:
result = qa_chain({"query": query})

print(f"\nQuestion: {query}")
print(f"Answer: {result['result']}")
print("\nRelevant source documents:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")  # Print first 200 characters of each document

### Example that demonstrates why we should use reranking 

In [123]:
chunks = [
    "The capital of France is great.",
    "The capital of France is huge.",
    "The capital of France is beautiful.",
    """Have you ever visited Paris? It is a beautiful city where you can eat delicious food and see the Eiffel Tower. 
    I really enjoyed all the cities in france, but its capital with the Eiffel Tower is my favorite city.""", 
    "I really enjoyed my trip to Paris, France. The city is beautiful and the food is delicious. I would love to visit again. Such a great capital city."
]
docs = [Document(page_content=sentence) for sentence in chunks]


def compare_rag_techniques(query: str, docs: List[Document] = docs) -> None:
    embeddings = OpenAIEmbeddings()
    vectorstore = FAISS.from_documents(docs, embeddings)

    print("Comparison of Retrieval Techniques")
    print("==================================")
    print(f"Query: {query}\n")
    
    print("Baseline Retrieval Result:")
    baseline_docs = vectorstore.similarity_search(query, k=2)
    for i, doc in enumerate(baseline_docs):
        print(f"\nDocument {i+1}:")
        print(doc.page_content)

    print("\nAdvanced Retrieval Result:")
    custom_retriever = CustomRetriever(vectorstore=vectorstore)
    advanced_docs = custom_retriever.get_relevant_documents(query)
    for i, doc in enumerate(advanced_docs):
        print(f"\nDocument {i+1}:")
        print(doc.page_content)


query = "what is the capital of france?"
compare_rag_techniques(query, docs)

Comparison of Retrieval Techniques
Query: what is the capital of france?

Baseline Retrieval Result:

Document 1:
The capital of France is great.

Document 2:
The capital of France is beautiful.

Advanced Retrieval Result:

Document 1:
I really enjoyed my trip to Paris, France. The city is beautiful and the food is delicious. I would love to visit again. Such a great capital city.

Document 2:
Have you ever visited Paris? It is a beautiful city where you can eat delicious food and see the Eiffel Tower. 
    I really enjoyed all the cities in france, but its capital with the Eiffel Tower is my favorite city.


## Method 2: Cross Encoder models

<div style="text-align: center;">

<img src="../images/rerank_cross_encoder.svg" alt="rerank cross encoder" style="width:40%; height:auto;">
</div>

### Define the cross encoder class

In [56]:
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

class CrossEncoderRetriever(BaseRetriever, BaseModel):
    vectorstore: Any = Field(description="Vector store for initial retrieval")
    cross_encoder: Any = Field(description="Cross-encoder model for reranking")
    k: int = Field(default=5, description="Number of documents to retrieve initially")
    rerank_top_k: int = Field(default=3, description="Number of documents to return after reranking")

    class Config:
        arbitrary_types_allowed = True

    def get_relevant_documents(self, query: str) -> List[Document]:
        # Initial retrieval
        initial_docs = self.vectorstore.similarity_search(query, k=self.k)
        
        # Prepare pairs for cross-encoder
        pairs = [[query, doc.page_content] for doc in initial_docs]
        
        # Get cross-encoder scores
        scores = self.cross_encoder.predict(pairs)
        
        # Sort documents by score
        scored_docs = sorted(zip(initial_docs, scores), key=lambda x: x[1], reverse=True)
        
        # Return top reranked documents
        return [doc for doc, _ in scored_docs[:self.rerank_top_k]]

    async def aget_relevant_documents(self, query: str) -> List[Document]:
        raise NotImplementedError("Async retrieval not implemented")



### Create an instance and showcase over an example

In [ ]:
# Create the cross-encoder retriever
cross_encoder_retriever = CrossEncoderRetriever(
    vectorstore=vectorstore,
    cross_encoder=cross_encoder,
    k=10,  # Retrieve 10 documents initially
    rerank_top_k=5  # Return top 5 after reranking
)

# Set up the LLM
llm = ChatOpenAI(temperature=0, model_name="gpt-4o")

# Create the RetrievalQA chain with the cross-encoder retriever
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=cross_encoder_retriever,
    return_source_documents=True
)

# Example query
query = "What are the impacts of climate change on biodiversity?"
result = qa_chain({"query": query})

print(f"\nQuestion: {query}")
print(f"Answer: {result['result']}")
print("\nRelevant source documents:")
for i, doc in enumerate(result["source_documents"]):
    print(f"\nDocument {i+1}:")
    print(doc.page_content[:200] + "...")  # Print first 200 characters of each document